In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
validation_data = pd.read_parquet('/content/drive/My Drive/pattern_recognition/lab_2/validation.parquet')
test_data = pd.read_parquet('/content/drive/My Drive/pattern_recognition/lab_2/test.parquet')

In [3]:
from PIL import Image
import io
import numpy as np

In [49]:
import cv2
import numpy as np

def resize_image(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Define the target size
    width, height = 224, 224

    # Resize the image
    image_resized = cv2.resize(image, (width, height))

    return image_resized

# path = 'IMG_20220630_171344.jpg'
# p = resize_image(path)
# p.shape


In [54]:
def resize_image_from_url(image_url):
    try:
        # Fetch image from URL
        response = requests.get(image_url)
        response.raise_for_status()  # Raise an exception for invalid responses

        # Read the image from response content
        image_bytes = BytesIO(response.content)
        image = cv2.imdecode(np.frombuffer(image_bytes.read(), np.uint8), cv2.IMREAD_COLOR)

        # Define the target size
        width, height = 224, 224

        # Resize the image
        image_resized = cv2.resize(image, (width, height))

        return image_resized
    except Exception as e:
        print(f"Error: {e}")
        return None

In [17]:
# Last attribute excludes the top (classification) layers of the model, leaving only the feature extraction layers.
resnet_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)

# Freezing layers (making them non-trainable)
for layer in resnet_model.layers:
  layer.trainable = False

94765736/94765736 [==============================] - 1s 0us/step


In [33]:
import tensorflow as tf
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
global_average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()



(1, 224, 224, 3)
1/1 [==============================] - 0s 183ms/step


In [ ]:
# Read the image
image = mpimg.imread(path)

# Display the image
plt.imshow(image)
plt.axis('off')  # Turn off axis
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img=p
plt.imshow(img)
plt.show()

In [37]:
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
import numpy as np###read from step1,2
vocabulary=np.load('./vocabulary (1).npy')
##output of CNN without the prediction layer or any additional FC layers size(2048,0)
images_features=np.load('./validation_features_after_pooling (1) (1).npy')
#read from tokeniszed captions
captions_dict=np.load('./validation_tokenized_captions.npy')

print(vocabulary)
print(images_features.shape,len(images_features))
print(captions_dict[1])

[['a' '1']
 ['.' '2']
 ['in' '3']
 ...
 ['non-working' '2727']
 ['startseq' '2728']
 ['endseq' '2729']]
(1000, 2048) 1000
[2728    1   13    3    1   28  121    7   39   89   90  258    3    4
  339    2 2729    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [38]:
# inv_dict={v:k for k,v in vocabulary.items()}

inv_dict={}
for i in range(0,len(vocabulary)):
  inv_dict[vocabulary[i][1]]=vocabulary[i][0]
dictionary={v:k for k,v in inv_dict.items()}


In [39]:

VOCAB_SIZE = len(vocabulary)
MAX_LEN=37
def generator(images, caption):
    n_samples = 0

    X = []
    y_in = []
    y_out = []

    for i in range(0,len(images_features)):
        for j in range (0,len(captions_dict[i])-1):

            X.append(images_features[i])
            in_seq= [captions_dict[i][:j]]
            out_seq = captions_dict[i][j]
            in_seq = pad_sequences(in_seq, maxlen=MAX_LEN, padding='post', truncating='post')[0]
            # out_seq = to_categorical(int(out_seq), num_classes=VOCAB_SIZE)[0]
            out = np.zeros(VOCAB_SIZE)
            out[out_seq-1]=1

            y_in.append(in_seq)
            y_out.append(out)



    return X, y_in, y_out

In [66]:
from keras.models import load_model

# Load the model
model = load_model("trained_lstm_new (1).h5")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 embedding_input (InputLaye  [(None, None)]               0         []                            
 r)                                                                                               
                                                                                                  
 dense_input (InputLayer)    [(None, 2048)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 64)             174656    ['embedding_input[0][0]']     
                                                                                                  
 dense (Dense)               (None, 64)                   131136    ['dense_input[0][0]']     

In [67]:
def predict(model,one_image_features):
  text_inp = ['startseq']
  tokenized=[]
  test_feature=[one_image_features]
  test_feature=np.array(test_feature)
  test_feature=test_feature.reshape(2048,)

  xin=[]
  xin.append(test_feature)
  xin=np.array(xin)

  # print(xin.shape,xin[0].shape)

  count = 0
  caption = ''
  while count < 20:
      count += 1


      encoded = []
      for i in text_inp:
          encoded.append(float(dictionary[i]))
          # print(dictionary[i])

      while len(encoded)!=37:
        encoded.append(0)

      yin=[]
      yin.append(encoded)

      yin=np.array(yin)
      # print(yin[0].shape)
      yin[0]=yin[0].reshape(37,)
      # print(yin[0].shape,yin[0])
      # print(xin[0].shape,xin[0])


      prediction =np.argmax(model.predict([xin, yin]))
      tokenized.append(prediction+1)

      sampled_word = inv_dict[str(prediction+1)]



      if sampled_word == 'endseq':
          break
      else:
        caption = caption + ' ' + sampled_word
        text_inp.append(sampled_word)
  return caption,tokenized


In [1]:
from nltk.translate.bleu_score import sentence_bleu

In [10]:
true_captions=[['a','man','is','standing','near','wall'],['a','man', 'is ','wearing','blue','shirt']]
predicted=['a','man', 'is ','in','blue','shirt','a','white','of']

sentence_bleu(true_captions,predicted,weights=(.25,0.25,0.25,0.25))

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


5.072841446586652e-78

In [74]:
for i in range(30,50):
  k,v=predict(model,images_features[i])

  print(i,k)

1/1 [==============================] - 0s 48ms/step
30  a man in a black shirt and a man in a black shirt is standing .
1/1 [==============================] - 0s 49ms/step
31  a man in a red and black shirt is standing on a a brown .
1/1 [==============================] - 0s 46ms/step
32  a man in a black shirt is standing in the water .
1/1 [==============================] - 0s 51ms/step
33  a black and white dog is running through the grass .
1/1 [==============================] - 0s 52ms/step
34  a man is running through a field .
1/1 [==============================] - 0s 82ms/step
35  a man in a black shirt and a man in a black dog and a man in a black shirt
1/1 [==============================] - 0s 50ms/step
36  a boy in a black shirt is jumping on a skateboard .
1/1 [==============================] - 0s 51ms/step
37  a biker in the air in the air .
1/1 [==============================] - 0s 52ms/step
38  a boy in a red and black and black shirt is standing on a white and white dog

In [30]:
captions=np.load('./grouped_validation_captions.npy')

In [38]:
captions[20]

array(['A black dog is looking through the fence .',
       'A brown dog runs along a fence .',
       'A dark brown dog is running along a fence outside .',
       'A large black dog runs along a fence in the grass .',
       'The brown greyhound dog walks on green grass and looks through a fence .'],
      dtype='<U162')

In [68]:
predicted_captions=[]
for i in range(0,len(images_features)):
  k,v=predict(model,images_features[i])
  predicted_captions.append(k)

1/1 [==============================] - 0s 48ms/step


KeyboardInterrupt: 

In [53]:
print(predicted_captions)

[' a woman in a dog and a black and white shirt and black on a blue .', ' a boy in a red .', ' a black and white dog and a red and dog in a .', ' a dog is running', ' a boy in a red and black and black shirt and .', ' a boy in a red shirt is .', ' a man in a white and white and white and white and white in a girl and man .', ' a girl in a red shirt and a blue and white and a girl in a the a girl .', ' a boy in a red shirt and black and red the on a group .', ' a man in a dog and black and white his white the .', ' a boy in a red shirt is brown a skateboard .', ' a man in a red shirt and red and white the ball .', ' a boy in a red shirt is running through water .', ' a girl in a red shirt and red in black the black a girl and girl', ' a boy in a black and black shirt and a red and shirt and a shirt and a shirt .', ' a boy in a black shirt is water .', ' a woman in a dog and black dog and black on a dog .', ' a black and white dog is running in a field .', ' a blue and white of a white a

In [69]:
one_gram=[]
two_gram=[]
three_gram=[]
for i in range(0,len(images_features)):
  true_captions=[]
  for j in range(0,len(captions[i])):
    true_captions.append(captions[i][j].split(' '))
  # print(true_captions)
  predicted=predicted_captions[i].split(' ')
  # print(predicted)
  k1=sentence_bleu(true_captions,predicted,weights=(1,0,0,0))
  k2=sentence_bleu(true_captions,predicted,weights=(0.5,0.5,0,0))
  k3=sentence_bleu(true_captions,predicted,weights=(0.33,0.33,0.33,0))
  one_gram.append(k1)
  two_gram.append(k2)
  three_gram.append(k3)

  # print(k1,k2)
print('1_gram',np.mean(one_gram))
print('2_gram',np.mean(two_gram))
print('3_gram',np.mean(three_gram))

NameError: name 'captions' is not defined

In [72]:
one_gram=[]
two_gram=[]
three_gram=[]
for i in range(0,len(images_features)):
  true_captions=[]
  for j in range(0,1):
    true_captions.append(captions[i][j].split(' '))
  # print(true_captions)
  predicted=predicted_captions[i].split(' ')
  # print(predicted)
  k1=sentence_bleu(true_captions,predicted,weights=(1,0,0,0))
  k2=sentence_bleu(true_captions,predicted,weights=(0.5,0.5,0,0))
  k3=sentence_bleu(true_captions,predicted,weights=(0.33,0.33,0.33,0))
  one_gram.append(k1)
  two_gram.append(k2)
  three_gram.append(k3)

  # print(k1,k2)
print('1_gram',np.mean(one_gram))
print('2_gram',np.mean(two_gram))
print('3_gram',np.mean(three_gram))

1_gram 0.22457661183266225
2_gram 0.06696976420524094
3_gram 0.019471172336605887


In [85]:
# Helper function to compute similarity
def compute_similarity(predicted_captions, captions):
    bleus=[]
    predicted=predicted_captions.split(' ')
    for j in range(0,len(captions)):
      t=captions[j].split(' ')
      bleus.append(sentence_bleu(true_captions,predicted,weights=(1,0,0,0)))
    return np.mean(bleus)
# Step 3: Compute Accuracy
threshold = 0.4 # Adjust as needed
correct_predictions = 0
for generated in predicted_captions:
    for ground_truth in true_captions:
        similarity = compute_similarity(generated, ground_truth)
        if similarity >= threshold:
            correct_predictions += 1
            break  # Move to the next generated caption
accuracy = correct_predictions / len(predicted_captions)
accuracy



0.008

In [59]:
def predict_image_given_path(path,model):
  p=resize_image(path)
  training_images = np.array([p])
  training_images = np.stack(training_images)
  training_images = training_images.astype('float32')
  training_images = preprocess_input(training_images)
  print(training_images.shape)
  my_image = resnet_model.predict(training_images)

  extracted_features = global_average_pooling_layer(my_image)
  k,v=predict(model,extracted_features)
  return k


In [71]:
# predict(model,extracted_features)
import requests
from io import BytesIO
path='download.jpg'
predict_image_given_path(path,model)

(1, 224, 224, 3)
1/1 [==============================] - 0s 49ms/step


' a black dog and a black dog are in the snow .'

In [ ]:
validation_loss, validation_accuracy = model.evaluate(validation_data)
print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)